In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import pickle
from sklearn.ensemble import GradientBoostingClassifier
from skimage.feature import hog
from sklearn.model_selection import train_test_split
import glob2
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler


import sys
sys.path.append("C:/xgboost/python-package")

import xgboost as xgb

In [2]:
### TODO: Tweak these parameters and see how the results change.
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (14, 14) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()


scaler = MinMaxScaler(feature_range=(0, 1), copy=True)

In [3]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [4]:
def getHog(feature_image, orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0):
    if hog_channel == 'ALL':
        hog_features = []
        for channel in range(feature_image.shape[2]):
            hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                orient, pix_per_cell, cell_per_block, 
                                vis=False, feature_vec=True))
        hog_features = np.ravel(hog_features)        
    else:
        hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                    pix_per_cell, cell_per_block, vis=False, feature_vec=True)
    # Append the new feature vector to the features list
    return hog_features

In [5]:
def cspace(image, color_space='RGB'):
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(image)    
    return feature_image

In [6]:
cars = glob2.glob('vehicles\**\*.png')
notcars = glob2.glob('non-vehicles\**\*.png')

In [7]:
def newSpace(image):
    R = scaler.fit_transform(image[:, :, 0])
    #B = scaler.fit_transform(image[:, :, 2])
    Y = scaler.fit_transform(cspace(image, color_space='YCrCb')[:, :, 0])
    L = scaler.fit_transform(cspace(image, color_space='HLS')[:, :, 0])
    return np.dstack((R, Y, L))
    
    

In [8]:
alldata = {}

## HOG Features
features = []
for file in cars:
    image = mpimg.imread(file)
    
    feature_image = newSpace(image)
    
    hog_features = getHog(feature_image, hog_channel=1)
    features.append(hog_features)
    
    

    
for file in notcars:
    image = mpimg.imread(file)
    
    feature_image = newSpace(image)
    
    hog_features = getHog(feature_image, hog_channel=1)
    features.append(hog_features)
    
alldata["hogfeatures"] = features

In [9]:
y = np.hstack((np.ones(len(cars)), np.zeros(len(notcars))))
alldata["label"] = y

for each in alldata.keys():
    alldata[each] = np.array(alldata[each])
    
features = np.array(features)
features.shape

(17760, 1764)

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in sss.split(features, y):
    
    X_train, X_test = features[train_idx], features[test_idx]
    
    y_train, y_test = y[train_idx], y[test_idx]

In [11]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [12]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'colsample_bytree':0.6}
watchlist  = [(dtrain,'train'), (dtest,'eval')]
num_round = 155
bst = xgb.train(param, dtrain, num_round, watchlist, early_stopping_rounds=10)

[0]	train-error:0.228604	eval-error:0.239583
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 10 rounds.
[1]	train-error:0.16174	eval-error:0.167511
[2]	train-error:0.146537	eval-error:0.150619
[3]	train-error:0.128308	eval-error:0.13232
[4]	train-error:0.117399	eval-error:0.118806
[5]	train-error:0.10663	eval-error:0.115146
[6]	train-error:0.098888	eval-error:0.105293
[7]	train-error:0.094313	eval-error:0.103885
[8]	train-error:0.085797	eval-error:0.10107
[9]	train-error:0.081855	eval-error:0.094032
[10]	train-error:0.076788	eval-error:0.091498
[11]	train-error:0.073902	eval-error:0.091216
[12]	train-error:0.070524	eval-error:0.086712
[13]	train-error:0.070242	eval-error:0.08643
[14]	train-error:0.06616	eval-error:0.081081
[15]	train-error:0.063626	eval-error:0.081363
[16]	train-error:0.0625	eval-error:0.080518
[17]	train-error:0.058488	eval-error:0.0808
[18]	train-error:0.057151	eval-error:0.077703
[

In [13]:
pred = bst.predict(dtest)
predictions = (pred > 0.5) * 1

from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.96283783783783783

In [14]:
bst.save_model("xgb.model")